任务链接：https://colab.research.google.com/drive/1Dx8hZXIBCHLbcamvE-vVzzcw0VqurPVz?usp=sharing

### 导入库（默认已经安装）


In [168]:
import json
import time

import requests

import pandas as pd
import re

### 从 Huggingface 爬下来数据：


In [169]:
start_time = time.time()

url = "https://datasets-server.huggingface.co/rows?dataset=AdaptLLM%2Ffinance-tasks&config=Headline&split=test&length=100&offset="  # 网页地址

data = []  # 存储数据
count = 0  # 用于储存页码
fails = []  # 存储失败的offset编号

# 因为huggingface给的api是分页的，需要循环到最后一个offset获取数据
for offset_num in range(0, 20500, 100):

    response = requests.get(url + str(offset_num))  # 组合url并请求数据

    if response.status_code == 200:
        data_this = response.json()
        data.append(data_this)
        count += 1
        print(f"已获取数据页面", count)  # 页码从1开始
    else:
        fails.append(offset_num)
        count += 1
        print(f"获取数据页面失败", count)

end_time = time.time()
time_used = end_time - start_time

print("数据爬取完成，此步骤共耗时", time_used, "秒。")
print("获取失败数据页码:", fails)

已获取数据页面 1
已获取数据页面 2
已获取数据页面 3
已获取数据页面 4
已获取数据页面 5
已获取数据页面 6
已获取数据页面 7
已获取数据页面 8
已获取数据页面 9
已获取数据页面 10
已获取数据页面 11
已获取数据页面 12
已获取数据页面 13
已获取数据页面 14
已获取数据页面 15
已获取数据页面 16
已获取数据页面 17
已获取数据页面 18
已获取数据页面 19
已获取数据页面 20
已获取数据页面 21
已获取数据页面 22
已获取数据页面 23
已获取数据页面 24
已获取数据页面 25
已获取数据页面 26
已获取数据页面 27
已获取数据页面 28
已获取数据页面 29
已获取数据页面 30
已获取数据页面 31
已获取数据页面 32
已获取数据页面 33
已获取数据页面 34
已获取数据页面 35
已获取数据页面 36
已获取数据页面 37
已获取数据页面 38
已获取数据页面 39
已获取数据页面 40
已获取数据页面 41
已获取数据页面 42
已获取数据页面 43
已获取数据页面 44
已获取数据页面 45
已获取数据页面 46
已获取数据页面 47
已获取数据页面 48
已获取数据页面 49
已获取数据页面 50
已获取数据页面 51
已获取数据页面 52
已获取数据页面 53
已获取数据页面 54
已获取数据页面 55
已获取数据页面 56
已获取数据页面 57
已获取数据页面 58
已获取数据页面 59
已获取数据页面 60
已获取数据页面 61
已获取数据页面 62
已获取数据页面 63
已获取数据页面 64
已获取数据页面 65
已获取数据页面 66
已获取数据页面 67
已获取数据页面 68
已获取数据页面 69
已获取数据页面 70
已获取数据页面 71
已获取数据页面 72
已获取数据页面 73
已获取数据页面 74
已获取数据页面 75
已获取数据页面 76
已获取数据页面 77
已获取数据页面 78
已获取数据页面 79
已获取数据页面 80
已获取数据页面 81
已获取数据页面 82
已获取数据页面 83
已获取数据页面 84
已获取数据页面 85
已获取数据页面 86
已获取数据页面 87
已获取数据页面 88
已获取数据页面 89
已获取数据页面 90
已获取数据页面 91
已获取数据页面 

In [ ]:
data

### 把 json 转换为 dataframe 进行编辑


In [170]:
all_data = []

# 因为上一步把数据存在了list里，现在单独提取出来
for item in data:
    for row in item['rows']:
        all_data.append(row['row'])

df = pd.DataFrame(all_data)  # 将json格式转换为dataframe格式
df

,gold_index,class_id,input,id,options
0,1,0,"Headline: ""Gold falls to Rs 30,800; silver dow...",0,"[No, Yes]"
1,0,7,Headline: february gold rallies to intraday hi...,1,"[No, Yes]"
2,0,5,Please answer a question about the following h...,2,"[No, Yes]"
3,1,3,"Read this headline: ""gold closes lower as doll...",3,"[No, Yes]"
4,0,1,"gold adds $42, or 2.4%, to trade at $1,833.30/...",4,"[No, Yes]"
...,...,...,...,...,...
20495,1,4,"""gold futures end $2 higher after two-session ...",20495,"[No, Yes]"
20496,0,2,Here__ what__ driving bitcoin buyers_ rush to ...,20496,"[No, Yes]"
20497,1,0,"Headline: gold rate today: gold, silver down i...",20497,"[No, Yes]"
20498,0,7,"oil turns lower, gold holds gain after jobs\nQ...",20498,"[No, Yes]"


### 清洗数据，删除影响分隔问题与答案的文本


In [171]:
start_time = time.time()

new_df = df.copy()  # 复制一份数据，避免直接修改元数据

# 指定需要删除的文本
patterns_to_remove = [
    r'Yes or No\?',  # 加入反斜杠转义，表示？也是需要删除的
    r'No or Yes\?',
    r'Options: - Yes - No',
    r'Options: - No - Yes',
    r'Question:',
    r'Answer:',
    r'Q:',
    r'A:',
    r'Headline:'
]

# 对每行文本进行处理，去除指定内容，使用enumerate同时获取原始索引和对应值（因为text被修改了两次）
for i, text in enumerate(new_df['input']):

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)  # 删除指定文本，无论大小写

    text = re.sub(r'[\'\"]', '', text)  # 删除所有引号
    new_df.loc[i, 'input'] = text  # 将处理后的文本存储回 DataFrame

end_time = time.time()
time_used = end_time - start_time

print("数据清理完成，此步骤共耗时", time_used, "秒。")

new_df

数据清理完成，此步骤共耗时 4.007127523422241 秒。


,gold_index,class_id,input,id,options
0,1,0,"Gold falls to Rs 30,800; silver down at Rs 41...",0,"[No, Yes]"
1,0,7,february gold rallies to intraday high of $90...,1,"[No, Yes]"
2,0,5,Please answer a question about the following ...,2,"[No, Yes]"
3,1,3,Read this gold closes lower as dollar rises\n...,3,"[No, Yes]"
4,0,1,"gold adds $42, or 2.4%, to trade at $1,833.30/...",4,"[No, Yes]"
...,...,...,...,...,...
20495,1,4,gold futures end $2 higher after two-session d...,20495,"[No, Yes]"
20496,0,2,Here__ what__ driving bitcoin buyers_ rush to ...,20496,"[No, Yes]"
20497,1,0,"gold rate today: gold, silver down in morning...",20497,"[No, Yes]"
20498,0,7,"oil turns lower, gold holds gain after jobs\n ...",20498,"[No, Yes]"


### 以问号分隔问题与答案，储存为 JSON 格式


In [173]:
import uuid  # 使用uuid模组生成唯一标识符（问题要求的uid）

start_time = time.time()


def get_qa(text):

    qa_pairs = []
    pairs = text.split('\n')    # 以换行符分隔文本

    for pair in pairs:
        if '?' in pair:
            question, answer = pair.split('?', 1)  # 仅以第一个问号分隔，避免影响到后续的问题中的问号

            if question and answer:
                qa_pairs.append({
                    "id": uuid.uuid4(),  # 使用 UUID 生成唯一标识符
                    "Question": question.strip(),  # 去除首尾空格，不然输出的文本前面会带/n/n
                    "Answer": answer.strip()
                })

    return qa_pairs


all_qa_pairs = new_df['input'].apply(get_qa)
print(all_qa_pairs[1])  # 查看第一行的问答对，已经符合要求

end_time = time.time()
time_used = end_time - start_time

print("数据分隔储存完成，此步骤共耗时", time_used, "秒。")

[{'id': UUID('a368b58f-7439-4bf5-b6ef-03f108e820f2'), 'Question': 'Does the news headline talk about price', 'Answer': 'Yes'}, {'id': UUID('68ba4979-950b-4d27-9f40-75f54c12393f'), 'Question': 'Does the news headline talk about price in the past', 'Answer': 'Yes'}, {'id': UUID('3ee6329c-389b-4222-8d18-f7002e970919'), 'Question': 'Does the news headline talk about price in the past', 'Answer': 'Yes'}, {'id': UUID('404b0c8f-f282-42a8-9938-db666d0fd9c7'), 'Question': 'Does the news headline talk about price in the future', 'Answer': 'No'}, {'id': UUID('d94da547-c6f2-4a4e-bff2-cf8d0aa2fe71'), 'Question': 'Does the news headline talk about price staying constant', 'Answer': 'No'}]
数据分隔储存完成，此步骤共耗时 0.8094985485076904 秒。


### 将所有的问答以 dataframe 形式呈现，一共有 80572 对问答


In [174]:
dfs = []

for qa_pairs in all_qa_pairs:
    df = pd.DataFrame(qa_pairs)
    dfs.append(df)

final_df = pd.concat(dfs, ignore_index=True)    # 连接所有 DataFrame，忽略原始索引

print("共有", len(final_df), "个成功提取的有效问答对")
final_df

共有 80572 个成功提取的有效问答对


,id,Question,Answer
0,50acacd4-8836-4a6d-919a-43cda019a8e9,"Gold falls to Rs 30,800; silver down at Rs 41,...",Yes
1,729f875a-5b42-4723-9d81-62fa26ec6cb1,gold futures add to gains after adp data Now a...,Yes
2,0f1ba72f-2594-4ca8-a2d5-2accec78fc76,Gold holds on to modest loss after data Now an...,No
3,17aa7ca5-6da5-4a37-a305-6187e7b3eb66,"spot gold quoted at $417.50, down 20c from new...",No
4,0f3276fe-752f-4c8d-b45a-c6b7837ea726,"gold hits new record high at $1,036.20 an ounc...",No
...,...,...,...
80567,75dd2131-2577-4730-8522-797c273457c1,Does the news headline talk about price stayin...,No
80568,dabe3fc0-ccc4-4d27-beb0-9a91063a4eee,Does the news headline talk about a general ev...,No
80569,236b4066-fa9e-434b-b9a0-4c59e3596f16,Does the news headline talk about price in the...,No
80570,7d932e07-1014-48a5-9140-cb96fb546766,Does the news headline talk about a general ev...,No
